#  Classificação Logística com Spark usando PySpark



## Inicializando o PySpark

In [2]:
import findspark
import pyspark
from pyspark.sql import SparkSession

# Faz a Interafce entre o Spark e o Jupyter Notebook
findspark.init()

# Inicializando uma Sessão no Spark
spark = SparkSession.builder.appName("Classificação Logística").getOrCreate()

### Hiper Parâmetros

> + **link**: define a função de link. Valores possíveis: identity, log, inverse, logit, probit, cloglog e sqrt
> + **maxIter**: número máximo de interações no treinamento do modelo. (padrão 100) 
> + **regParam**: valor de regularização. (padrão 0)



---

## Aplicando  Classificação Logística ao DataSet Churn



### Carregando o Cunjunto de Dados Carros

In [4]:
churn = spark.read.csv("../Material_do_Curso/Churn.csv",
                             header=True, inferSchema=True, sep=";")
print(f"Quantidade de Registros do Dataset: {churn.count()}")
churn.show(5, truncate=True)

Quantidade de Registros do Dataset: 10000


+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|CreditScore|Geography|Gender|Age|Tenure| Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|        619|   France|Female| 42|     2|       0|            1|        1|             1|       10134888|     1|
|        608|    Spain|Female| 41|     1| 8380786|            1|        0|             1|       11254258|     0|
|        502|   France|Female| 42|     8| 1596608|            3|        1|             0|       11393157|     1|
|        699|   France|Female| 39|     1|       0|            2|        0|             0|        9382663|     0|
|        850|    Spain|Female| 43|     2|12551082|            1|        1|             1|         790841|     0|
+-----------+---------+------+---+------+--------+-------------+---------+--------------+-------

### Escolha das Variáveis Independentes e Dependente para treinamento do Modelo

Para esse exemplo, as variáveis dependentes serão os atritutos  **Consumo | Cilindros | Cilindradas |** e a variável Dependente será o Atributo **| HP |**

ou seja, vamos usar as variáveis independentes para prever a dependente, para isso, vamos treinar o modelo com os dados do dataset.

### Importação do Módulo do PySpark Para o Pré-Processamento dos Dados

In [6]:
from pyspark.ml.feature import RFormula

In [7]:
# Colocando os dados no Formato de dataframe do Spark usando o RFormula
rformula = RFormula(formula="Exited ~ .",
                    featuresCol="independente",
                    labelCol="dependente")
churn_rf = rformula.fit(churn).transform(churn)
churn_rf.select("independente", "dependente").show(5, truncate=False)

+--------------------------------------------------------------+----------+
|independente                                                  |dependente|
+--------------------------------------------------------------+----------+
|[619.0,1.0,0.0,0.0,42.0,2.0,0.0,1.0,1.0,1.0,1.0134888E7]      |1.0       |
|[608.0,0.0,0.0,0.0,41.0,1.0,8380786.0,1.0,0.0,1.0,1.1254258E7]|0.0       |
|[502.0,1.0,0.0,0.0,42.0,8.0,1596608.0,3.0,1.0,0.0,1.1393157E7]|1.0       |
|(11,[0,1,4,5,7,10],[699.0,1.0,39.0,1.0,2.0,9382663.0])        |0.0       |
|[850.0,0.0,0.0,0.0,43.0,2.0,1.2551082E7,1.0,1.0,1.0,790841.0] |0.0       |
+--------------------------------------------------------------+----------+
only showing top 5 rows



### Separando os Dados entre conjunto de Treino e Teste

O Conjunto de dados serão separados entre treino e teste, sendo que **80%** será para treinar o modelo e **20%** para testar o modelo.

In [8]:
churn_train, churn_test = churn_rf.randomSplit([0.8, 0.2])
print(f"Quantidade de Dados de Treino: {churn_train.count()}")
print(f"Quantidade de Dados de Teste: {churn_test.count()}")

Quantidade de Dados de Treino: 8008


Quantidade de Dados de Teste: 1992


---

### Importação do Módulo do PySpark Para Criação do Modelo de Classificação Logística

In [9]:
from pyspark.ml.classification import LogisticRegression

#### Instanciando Objeto e criando o Modelo

In [10]:
# Instanciando o objeto LinearRegression
obj_lr = LogisticRegression(featuresCol="independente",
                            labelCol="dependente",
                            maxIter=1000, standardization=True)

# Criando o Modelo
model_lr = obj_lr.fit(churn_train)

23/03/04 23:46:14 WARN BlockManager: Asked to remove block broadcast_44, which does not exist


### Parâmetros do Modelo Treinado

In [11]:
resumo = model_lr.summary
acuracia = resumo.accuracy
precisao = resumo.weightedPrecision
recall = resumo.weightedRecall
auc = resumo.areaUnderROC

print(f"Acurácia: {acuracia}")
print(f"Presisão: {precisao}")
print(f"recall: {recall}")
print(f"Área Sobre a curva: {auc}")

Acurácia: 0.8124375624375625
Presisão: 0.782087319536303
recall: 0.8124375624375624
Área Sobre a curva: 0.7655822818870617


#### Realizando Presição com o Modelo Criado

In [14]:
previsao_test = model_lr.transform(churn_test)
previsao_test.select("dependente", "prediction", "probability", "rawPrediction").show(5, truncate=False)

+----------+----------+----------------------------------------+--------------------------------------------+
|dependente|prediction|probability                             |rawPrediction                               |
+----------+----------+----------------------------------------+--------------------------------------------+
|1.0       |0.0       |[0.8437003172962992,0.15629968270370076]|[1.6860221498699997,-1.6860221498699997]    |
|1.0       |0.0       |[0.7521793673290733,0.2478206326709267] |[1.110269586762303,-1.110269586762303]      |
|1.0       |1.0       |[0.49778895042881327,0.5022110495711867]|[-0.008844255934740364,0.008844255934740364]|
|1.0       |0.0       |[0.847871693250834,0.152128306749166]   |[1.718005031091091,-1.718005031091091]      |
|1.0       |0.0       |[0.7505473314354185,0.24945266856458148]|[1.1015335235778099,-1.1015335235778099]    |
+----------+----------+----------------------------------------+--------------------------------------------+
only showi

### Avaliando o Modelo

In [21]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [23]:
avaliar = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction",
                                        labelCol="dependente",
                                        metricName="areaUnderROC")

areaUnderRoc = avaliar.evaluate(previsao_test)

print(f"areaUnderRoc: {areaUnderRoc}")

areaUnderRoc: 0.7734546915379094


---